In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
def msg(role, text):
    return {"role":role,"content": text }

In [4]:
sys_prompt = """당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. 근거:, E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.

근거: 해당 뉴스 기사에 대해서 ESG 점수를 작성한 근거를 작성합니다.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

답변 형식 예시는 다음과 같습니다.
근거: 이 뉴스 기사는 방화6구역 조합과 HDC현대산업개발 간의 공사비 갈등과 시공사 계약 해지 과정을 다루고 있습니다. 주 내용은 공사비 인상 문제로 인한 갈등이 심화되어 결국 시공사 교체를 논의하는 상황에 놓여 있다는 것입니다. 이로 인해 사회적 이슈로는 공사 중단 사태가 장기화되고 법적 분쟁까지 이어졌습니다. 환경에 대한 명시적 내용은 없으며, 지배구조와 관련되어 시공사와 조합 간의 협상 실패와 조합원 반발이 드러나고 있습니다.
E: 2
S: 1
G: 2
"""

user_prompt = """한국기업거버넌스포럼이 "한화그룹이 오너가 3세인 3형제의 입장에서 인적 분할을 결정했다"고 지적했다.

포럼은 19일 '김동관 부회장, 김동원 사장, 김동선 부사장에게 드리는 5가지 질문'이라는 논평을 통해 "일반 주주는 배제됐다"며 이같이 밝혔다.

앞서 ㈜한화는 14일 기존 사업 중심의 존속 법인과 테크 및 라이프 부문이 포함된 신설 법인으로 분할하는 계획을 발표했다.

분할 비율은 존속 법인 76%, 신설 법인 24%다. 6월 임시 주주총회 등을 거쳐 7월 중 분할을 마무리한다.
포럼은 "㈜한화 주가는 분할 발표 전 8거래일 간 26% 상승한 후 공시 후 3거래일 간 추가로 22% 급등했다"고 말했다.

그러면서 "시총은 2년 간 5배 가까이 증가했다"면서 "거버넌스 우려 등으로 국내 지주사 중 할인율이 가장 심하다"고 짚었다.

이번 인적 분할 역시 홀로서기에 나서는 김동선 부사장이 아닌 일반 주주 관점에서는 '명백히 더 나은 대안'을 선택하지 않았다는 비판이다.

포럼 측은 "기존 4개(방산·우주항공, 조선·해양, 에너지·케미컬, 금융) 사업과 신설 2개(테크 솔루션, 라이프 솔루션), 자체 사업 2개(건설, 글로벌) 등 사업군 별로 지주사를 분리해야 한다"고 주장했다.

이어 "MSCI 산업 분류도 상이하고 회사 측이 제시한 장기 성장률도 큰 편차를 보인다"며 "일반 주주 이익 침해로 연결된다"고 설명했다.

포럼은 또 "공시나 자료 어디에도 다른 대안을 충분히 검토하고 어떤 이유로 이번 분할 대안을 선택했는 지에 대한 설명이 없다"고도 했다.

분할 비율을 존속법인 76%, 신설 법인 24%로 책정한 데 대해서도 의문을 제기했다.

포럼 측은 "신설 법인에 부채 이관이 없고 1년 운영 자금으로 현금 1,000억원을 배분한 것은 분할 비율의 적정성 여부에 의문을 남긴다"고 분석했다.

이사회의 전문성이 떨어지는 점에 대해서도 목소리를 높였다.

포럼은 "현재 한화 및 존속 법인 4명의 독립이사는 비즈니스 경험이 없는 철학과 교수, 검사·변호사 출신 등으로 구성됐다"고 강조했다.

끝으로 "경험 및 지식, 자본 시장 및 기업 거버넌스에 대한 이해가 있는 독립적인 인물 중심으로 이사회를 재구성해야 한다"고 촉구했다."""

In [5]:
# 프롬프트 생성
res = client.responses.create(
    model='gpt-4o-mini'
    , input=[ msg("system", sys_prompt), msg("user", user_prompt)]
    # , input=[{"role":"system", "content":sys_prompt}
    #        , {"role":"user", "content":user_prompt}]
    , temperature=0.5
)
res.output_text

'근거: 이 뉴스 기사는 한화그룹의 인적 분할 결정과 관련하여 한국기업거버넌스포럼의 비판을 다루고 있습니다. 주주들이 배제된 결정 과정과 이사회의 전문성 부족, 일반 주주 이익 침해에 대한 우려가 강조되었습니다. 이는 지배구조의 투명성과 책임 있는 경영에 대한 문제를 나타내며, 사회적 측면에서도 일반 주주를 고려하지 않은 결정이 부정적 영향을 미칠 수 있음을 시사합니다. 환경에 대한 언급은 없으며, 전반적으로 지배구조와 사회적 책임에 대한 비판이 두드러집니다.\nE: 1\nS: 2\nG: 1'

In [7]:
sys_prompt1 = """당신은 ESG에 대해서 점수를 계산하는 모델입니다. 평가 기준은 다음과 같습니다.

## 지시사항
주어진 평가 기준을 바탕으로 뉴스 기사로부터 환경, 사회, 지배 구조에 대해서 점수를 측정하십시오.

## 평가 기준
환경
1점: 환경 오염 유발, 법적 제재, 규제 위반 등의 사건이 발생한 경우. 예: 심각한 오염 사고, 규제 위반으로 인한 벌금 부과
2점: 환경 보호에 대한 소극적인 태도 또는 일회성 활동에 그치는 경우. 예: 임시적인 환경 캠페인 참여, 적극적인 감축 목표 미설정
3점: 환경 보호를 위한 노력이 있으나 체계적인 전략이 부족한 경우. 예: 온실가스 감축 계획 발표, 일부 친환경 인증 획득
4점: 체계적인 환경 관리 계획이 있으며, 목표 달성에 꾸준히 투자하는 경우. 예: 지속 가능한 에너지 사용, 체계적인 폐기물 관리 프로그램 운영
5점: 적극적인 환경 정책과 뛰어난 실적 결과를 보이며 업계 선도적 활동을 하는 경우. 예: 탄소 배출 네트 제로(Net Zero) 목표 실현, 친환경 기술 선도

사회
1점: 인권, 근로자 안전 문제 발생, 지역 사회와의 갈등 등 부정적 이슈 발생. 예: 직원 착취 문제, 안전 관리 소홀로 인한 사고
2점: 직원 복지나 지역 사회 기여에 대한 제한적 활동 또는 단기적인 참여. 예: 일회성 기부, 사내 복지 프로그램 부족
3점: 직원 복지 및 사회적 기여에 관심을 가지며 일부 프로그램을 운영하는 경우. 예: 사내 복지 프로그램 운영, 특정 지역 사회 활동 참여
4점: 직원 복지, 다양성, 지역 사회 공헌에 적극적으로 나서며 책임감 있게 운영하는 경우. 예: 다양성 강화 프로그램, 정기적인 사회공헌 활동
5점: 사회적 가치 창출에 지속적으로 기여하며 업계 모범적인 활동을 보여주는 경우. 예: 폭넓은 사회적 책임 활동, 소외 계층 지원 프로그램

지배구조
1점: 지배구조의 투명성 부족, 부패 사건 또는 규제 위반이 발생한 경우. 예: 부패 스캔들, 재무 조작
2점: 지배구조 개선 노력이 부족하며, 외부 감사나 투명성 문제에 있는 경우. 예: 의사 결정 과정 불투명, 주주와의 소통 부족
3점: 기본적인 지배구조 원칙을 따르나, 일부 개선의 여지가 있는 경우. 예: 주주와의 소통 강화 중, 투명성 개선 노력
4점: 투명하고 윤리적인 지배구조를 실천하고 있으며, 책임 있는 경영을 하는 경우. 예: 정기적인 외부 감사, 투명한 의사결정 구조
5점: 업계에서 모범이 되는 윤리적 지배구조를 유지하고 지속적 개선을 보이는 경우. 예: 독립적 이사회 구성, 주주 및 이해관계자와의 적극적 소통

## 답변 형식
답변 형식은 반드시 아래의 형식을 따르십시오. 근거:, E:, S:, G: 외에는 그 어떠한 문장도 덧붙이지 마십시오.

근거: 해당 뉴스 기사에 대해서 ESG 점수를 작성한 근거를 작성합니다.
E: 1~5 사이의 정수
S: 1~5 사이의 정수
G: 1~5 사이의 정수

답변 형식 예시는 다음과 같습니다.
근거: 이 뉴스 기사는 방화6구역 조합과 HDC현대산업개발 간의 공사비 갈등과 시공사 계약 해지 과정을 다루고 있습니다. 주 내용은 공사비 인상 문제로 인한 갈등이 심화되어 결국 시공사 교체를 논의하는 상황에 놓여 있다는 것입니다. 이로 인해 사회적 이슈로는 공사 중단 사태가 장기화되고 법적 분쟁까지 이어졌습니다. 환경에 대한 명시적 내용은 없으며, 지배구조와 관련되어 시공사와 조합 간의 협상 실패와 조합원 반발이 드러나고 있습니다.
E: 2
S: 1
G: 2
"""
user_prompt1 = """한화시스템이 새해 병오년을 맞아 협력사와의 동반성장과 상생협력을 한 단계 끌어올리기 위한 비전을 제시했다. 최근 김승연 한화그룹 회장이 신년사에서 한화 상생 경영의 원칙인 '함께 멀리'의 중요성을 다시 한 번 강조한 데 따른 행보다. 한화시스템(대표이사 손재일)은 16일 서울 중구 더플라자호텔에서 '2026년 동반성장데이'를 개최했다고 밝혔다. 올해로 4회차를 맞은 이번 행사는 손재일 대표이사를 비롯한 한화시스템 임직원 30여명, 주요 협력사 대표이사 70여명 등 역대 가장 많은 인원이 참석한 가운데 진행됐다. 이 자리에서 한화시스템은 협력사에 △글로벌 동반 진출 방안 모색 △미래 기술 확보 지원 △안전·공정·상생 최우선 등 동반성장 강화를 위한 3가지 약속을 제시했다. 이를 통해 방산·ICT 분야에서 대한민국 대·중·소 기업이 함께 성장할 수 있는 지속가능한 생태계를 선도하겠다는 목표를 세웠다. 한화시스템은 올해 미국·유럽·중동 등 글로벌 시장에 자사와 협력사의 기술 및 제품이 함께 진출할 수 있도록 해외 판로 개척 방안과 수출 협업 기회를 적극 모색할 계획이다. 또한 AI·무인체계 등 미래 유망기술 분야의 공동 기술협력과 연구개발(R&D) 지원을 확대해, 협력사가 경쟁력을 갖춘 강소기업으로 성장할 수 있도록 중장기적으로 뒷받침한다. 앞서 한화시스템은 지난 3년간 투자지원 및 기술협업 과제를 통해 협력사와 함께 △함정용 전원분배장치 표준모델 △국산 다기능레이다 로타리조인트 등의 기술 개발을 완료하고 사업화한 바 있다. 이외에도 한화시스템의 협력사 경영역량 강화 지원 프로그램인 HPMS(Hanwha Systems Partner Management Support package program) 참여 기업을 기존 대비 300% 확대하고, 공정거래협약 체결 대상을 120개사까지 늘리는 등 협력사들의 인권경영 및 공정거래 지원에도 최선을 다할 것을 약속했다. HPMS는 한화시스템이 협력사의 경영 역량 강화를 위해 개발한 통합형 경영지원 프로그램. 기존에 별도로 운영되던 △경영진단 △경영자문 △ESG 진단 컨설팅 프로그램이 하나로 통합된 순환형 프로그램이다. 손재일 한화시스템 대표이사는 "지난해 한화시스템의 국내사업 수주 및 해외수출 성과는 협력사 임직원 분들의 품질개선을 위한 노고와 납기 준수를 위한 헌신과 노력이 있었기에 가능했다"며 "올해도 세계 시장을 향해 함께 나아갈 수 있도록 기술협력·ESG·인재육성 전반에서 파트너사에 대한 지원을 아끼지 않을 것"이라고 말했다. 동반성장데이는 한화시스템이 협력사들과 함께 지난 한 해의 동반성장 정책과 상생협력의 주요 성과를 되돌아보고 신년 계획을 마련하는 행사로, 2022년 이후 매년 개최됐다. 2022년도부터 연말에 개최해오던 행사를 올해는 새해의 포문을 열며 1월에 열게 됐다. 한화시스템은 매해 '파트너스 데이' 행사를 통해 협력사 임직원들에게 식사·간식·필요용품 등을 제공하고, '상생협력 간담회'를 통해 협력사의 의견을 적극 청취하고 수렴하는 등 매해 파트너사를 위한 다양한 지원 및 소통 활동을 펼치고 있다. 한편, 한화시스템은 지난해 글로벌 ESG 평가기관 모건스탠리캐피털인터내셔널(MSCI)로부터 국내 방산기업 중 처음으로 최고 등급인 'AAA'를 받았으며, 한국ESG기준원(KCGS)의 2025년 ESG 평가에서도 5년 연속 통합 A등급을 획득한 바 있다. """ 



In [8]:
# 프롬프트 생성
res = client.responses.create(
    model='gpt-4o-mini'
    , input=[ msg("system", sys_prompt1), msg("user", user_prompt1)]
    # , input=[{"role":"system", "content":sys_prompt}
    #        , {"role":"user", "content":user_prompt}]
    , temperature=0.5
)
res.output_text

'근거: 이 뉴스 기사는 한화시스템이 협력사와의 동반성장 및 상생협력을 위한 비전을 제시하고, 다양한 지원 프로그램과 글로벌 시장 진출 계획을 발표한 내용을 다루고 있습니다. 특히, ESG 경영에 대한 강조와 함께 협력사 인권경영 및 공정 거래 지원을 위한 구체적인 약속이 포함되어 있습니다. 이러한 점에서 환경, 사회, 지배구조 모두 긍정적인 평가를 받을 수 있습니다.\nE: 4\nS: 5\nG: 5'

In [16]:
sys_prompt2 = """당신은 글로벌 대기업 인사(HR) 평가 엔진이다. 목표는 자기소개서 텍스트를 기반으로 '내용 + 형식'을 구조화해 점수화하고, 근거 인용과 개선안을 제공하는 것이다. 평가는 경영학/조직행동 이론(신호이론, 직무적합(P-J), 조직적합(P-O), ASA, AMO, 목표설정, 인상관리, 구조화평가/BARS)을 실제 측정 규칙으로 번역해 적용한다.

[입력]
- essay: 자기소개서 본문(문자열)
- job: 채용 직무 정보(없을 수 있음)
- org_values: 회사 가치/인재상(없을 수 있음)
- constraints: 글자수/문항/경력구분 등(없을 수 있음)

[작동 규칙: 단계별 추론(내부 전용)]
1) 내부적으로 다음을 순서대로 수행하라:
   (a) 글 구조 파악
   (b) 핵심 주장/사례/성과 추출
   (c) STAR 구성요소 식별(상황-과제-행동-결과)
   (d) 직무/가치와의 연결 추정
   (e) 모듈별 BARS 채점
   (f) 리스크 탐지
   (g) 개선안 생성
2) 단, 위 내부 추론의 중간 풀이(사고 과정)는 절대 출력하지 말고, 최종 결과(JSON)만 출력하라.

[자기 일관성(Self-Consistency) 절차(내부 전용)]
- 서로 독립적인 평가 3회를 수행하라(관점만 다르게: 엄격/중립/관대).
- 각 모듈 점수는 중앙값(median)으로 확정하라.
- 3회 평가에서 모듈 점수 편차가 2점 이상(5점 척도)이면 '일관성_경고=true'로 표시하고,
  불일치 원인을 2줄로 요약하라(사고과정이 아니라 관찰된 불일치 요인만).

[평가 모듈 및 BARS(1~5점)]
A. 내용
- 직무적합: 직무 요구역량(KSA)과 경험/성과의 매칭
- 조직적합: 가치/협업/의사결정 방식의 정합
- AMO: 능력(스킬·학습), 동기(동기·지속성), 기회(자원·협업·프로세스 활용) 근거
- 성과임팩트: 결과의 정량성/전후 비교/기여도
- 증거품질: 주장 대비 행동·의사결정·갈등처리·성과의 구체성(STAR)
- 신뢰리스크: 과장/모순/검증불가/비밀유출/비방/책임전가

B. 형식
- 구조: 문단 역할/논리 흐름/질문 대응성
- 가독성: 한 문장 과다 길이, 복잡도, 핵심 선명도(정보처리 비용)
- 간결성: 중복/군더더기/모호어 비율
- 문장표현: 맞춤법/문장력/전문용어 남용
- 톤준수: 차별·공격·윤리 리스크 표현

[출력 언어 규칙]
- 출력 JSON의 모든 키는 한국어로 작성하라.
- 문자열 값은 원칙적으로 한국어로 작성하라. (직무명/고유명사/영문 약어는 원문 그대로 유지 가능)
- JSON을 유니코드 이스케이프(\\uXXXX)로 출력하지 말고, 한글을 그대로 출력하라.
- 반드시 'JSON만' 출력하라(설명 문장/마크다운/주석 금지).

[출력 형식: JSON만]
{
  "종합점수_100": 0-100,
  "점수_1to5": {
    "내용": {
      "직무적합": 1-5,
      "조직적합": 1-5,
      "AMO": 1-5,
      "성과임팩트": 1-5,
      "증거품질": 1-5,
      "신뢰리스크": 1-5
    },
    "형식": {
      "구조": 1-5,
      "가독성": 1-5,
      "간결성": 1-5,
      "문장표현": 1-5,
      "톤준수": 1-5
    }
  },
  "핵심강점": ["강점 3~5개(짧게)"],
  "핵심약점": ["약점 3~5개(짧게)"],
  "근거인용": {
    "긍정": [{"모듈": "성과임팩트", "인용": "essay 원문에서 그대로"}],
    "부정": [{"모듈": "간결성", "인용": "essay 원문에서 그대로"}]
  },
  "이론적용": [
    {
      "이론": "신호이론(Signaling theory)",
      "적용모듈": ["성과임팩트", "증거품질"],
      "적용설명": "측정 규칙 1~2문장"
    },
    {
      "이론": "직무적합(P-J fit)",
      "적용모듈": ["직무적합"],
      "적용설명": "측정 규칙 1~2문장"
    },
    {
      "이론": "조직적합(P-O fit) / ASA",
      "적용모듈": ["조직적합", "신뢰리스크"],
      "적용설명": "측정 규칙 1~2문장"
    },
    {
      "이론": "AMO(능력-동기-기회)",
      "적용모듈": ["AMO"],
      "적용설명": "측정 규칙 1~2문장"
    },
    {
      "이론": "목표설정이론(Goal-setting)",
      "적용모듈": ["성과임팩트", "증거품질"],
      "적용설명": "측정 규칙 1~2문장"
    },
    {
      "이론": "인상관리(Impression management)",
      "적용모듈": ["신뢰리스크", "문장표현"],
      "적용설명": "측정 규칙 1~2문장"
    },
    {
      "이론": "구조화평가/BARS",
      "적용모듈": ["모든모듈"],
      "적용설명": "5점 앵커 기반 평정(행동/근거 기준으로 점수 부여)"
    }
  ],
  "개선제안": [
    {
      "대상": "문장/문단 요약",
      "문제": "",
      "개선": "",
      "예시_바꿔쓰기": ""
    }
  ],
  "일관성_경고": true/false,
  "일관성_메모": "편차가 큰 경우에만 2줄 요약, 아니면 빈 문자열"
}

[추가 규칙]
- job/org_values가 없으면, 일반 대기업 기준으로 평가하되 그 가정을 '개선제안' 또는 '이론적용'의 설명에 짧게 남겨라.
- 보호특성(성별/나이/출신 등) 추정/활용 금지. 해당 내용이 평가에 들어오면 '톤준수' 또는 '신뢰리스크'에 플래그로만 반영.
- '근거인용'의 인용문은 반드시 essay 원문에서 그대로 가져와라(날조 금지).
- 출력의 모듈명은 반드시 아래 표준값 중 하나만 사용하라:
  [직무적합, 조직적합, AMO, 성과임팩트, 증거품질, 신뢰리스크, 구조, 가독성, 간결성, 문장표현, 톤준수]
"""

user_prompt2 = """저는 ‘사람의 의사결정이 더 나아지게 만드는 데이터’에 관심이 많습니다. 대학 시절 학생회에서 신입생 적응 프로그램을 운영하며, 참가자 만족도 설문을 모아도 다음 기획에 반영되지 않는 현실을 보았습니다. 그때부터 “좋은 의도”를 “좋은 결과”로 바꾸려면 관찰 가능한 근거와 실행 가능한 프로세스가 필요하다는 점을 배웠고, 이를 채용·평가 업무에 적용해보고 싶어 지원합니다.

첫 번째 경험은 동아리 리더로서 갈등을 해결했던 사례입니다. 대회 준비 기간에 12명의 팀이 전술을 두고 두 파로 갈렸고, 훈련 분위기가 급격히 냉각되었습니다(상황). 제 과제는 성과를 유지하면서도 팀이 다시 한 방향으로 움직이게 만드는 것이었습니다(과제). 저는 감정적 설득 대신, 각 전술의 장단점을 ‘상대 분석–역할 분담–리스크’ 관점으로 비교하는 미팅을 제안했고, 팀원들이 직접 근거를 제시하도록 회의 규칙을 만들었습니다(행동). 논의가 길어질 때는 핵심 쟁점을 칠판에 요약하고, 합의가 어려운 항목은 “다음 연습 경기에서 A안, 그다음 경기에서 B안”으로 실험 순서를 정해 결과로 판단하게 했습니다. 그 결과 훈련 집중도가 회복되었고, 대회 직전에는 팀원들과 상대팀을 분석하며 경기 전략을 보완할 수 있었습니다(결과). 이 과정에서 저는 목표 달성에는 개인의 적극적 참여와 신뢰 기반 소통이 필수임을 깨달았습니다.

두 번째 경험은 윤리 기준을 지키는 선택을 했던 일입니다. 하반기 사회적 거리두기 강화로 5인 이상 모임이 금지된 시기에, 한 선배가 “조용히 하면 괜찮다”며 단체 훈련을 비밀리에 추진하는 것을 알게 되었습니다. 저는 팀의 실적도 중요하지만 규정 위반이 가져올 위험이 더 크다고 판단했고, 공개적으로 문제를 제기하면 관계가 깨질 수 있어 먼저 선배와 1:1로 대화를 요청해 ‘리스크–대안–책임’ 순서로 설득했습니다. 동시에 온라인 전술 회의, 개인별 체력 루틴 공유, 소규모 분산 연습으로 목표를 대체할 방법을 제시했습니다. 결과적으로 단체 훈련은 중단되었고, 팀은 갈등 없이 준비를 이어갈 수 있었습니다. 이 경험은 제게 “성과를 내는 방식” 자체가 조직의 신뢰를 만든다는 기준을 남겼습니다.

이후 저는 사람과 조직의 문제를 더 구조적으로 보고 싶어 인사 데이터 프로젝트에 참여했습니다. 인턴 기간 동안 채용 서류 검토가 담당자마다 기준이 달라 시간이 많이 소요되는 것을 보며, 공통 언어가 되는 평가 프레임이 필요하다고 느꼈습니다. 저는 지원서 문장에서 STAR 요소를 표시하고, 직무역량 키워드와 연결해 요약하는 템플릿을 만들었고, 간단한 스프레드시트 규칙으로 중복 표현과 모호어(“열심히”, “최선을”)를 표시했습니다. 완벽하지는 않았지만 팀원들이 같은 기준으로 토론하는 데 도움이 되었고, 특히 “왜 이 후보가 좋은가”를 근거 문장으로 설명하는 습관이 생겼습니다. 협업과 소통이 좋아졌고, 협업과 소통이 좋아졌다는 피드백을 들었습니다.

한편 저는 보고서나 자기소개서를 쓸 때 ‘근거를 충분히 담겠다’는 욕심으로 문장이 길어지는 편이라, 읽는 사람이 한 번에 요지를 잡기 어렵다는 지적을 받은 적이 있습니다. 그래서 최근에는 문단의 첫 문장에 결론을 두고, 뒤에 근거를 붙이는 방식으로 다시 쓰는 훈련을 하고 있으며, 같은 내용을 다른 표현으로 반복하지 않도록 검토 체크리스트를 사용합니다. 이러한 개선 습관은 빠른 판단이 필요한 채용 환경에서 특히 중요하다고 생각합니다. 또한 여러 부서가 참여하는 프로젝트에서는 용어가 달라 오해가 생기기 쉬워, 회의마다 정의(Definition)와 결정사항(Decision)을 남기는 습관을 들였고, 최근 3개월간 30건의 지원서 샘플을 기준으로 템플릿을 보완했습니다.

저는 귀사의 채용 과정에서도 ‘신호의 질’을 높이고 싶습니다. 구체적 행동과 결과, 일관된 가치, 직무 관련 역량이 드러나는 문서가 더 빠르게 식별되면 평가자는 더 중요한 판단(면접 질문 설계, 역량 검증)에 시간을 쓸 수 있습니다. 저는 데이터 기반으로 평가 기준을 명확히 하되, 숫자만으로 사람을 단정하지 않는 균형을 지키겠습니다. 입사 후에는 직무별 역량 사전과 문항별 평가 기준을 정리하고, 지원서 평가 결과가 실제 성과와 어떤 관계가 있는지 검증하며 개선하는 HR 시스템을 만들고 싶습니다."""



In [17]:
res = client.responses.create(
    model="gpt-4o-mini",
    input=[msg("system", sys_prompt2), msg("user", user_prompt2)],
    temperature=0.5,
)

print(res.output_text)  # ✅ 줄바꿈이 실제로 적용되어 출력됨

{
  "종합점수_100": 85,
  "점수_1to5": {
    "내용": {
      "직무적합": 4,
      "조직적합": 4,
      "AMO": 4,
      "성과임팩트": 4,
      "증거품질": 4,
      "신뢰리스크": 3
    },
    "형식": {
      "구조": 4,
      "가독성": 3,
      "간결성": 3,
      "문장표현": 4,
      "톤준수": 5
    }
  },
  "핵심강점": ["구체적인 사례 제시", "문제 해결 능력", "데이터 기반 접근"],
  "핵심약점": ["문장 길이", "중복 표현", "가독성 저하"],
  "근거인용": {
    "긍정": [{"모듈": "성과임팩트", "인용": "훈련 집중도가 회복되었고, 대회 직전에는 팀원들과 상대팀을 분석하며 경기 전략을 보완할 수 있었습니다."}],
    "부정": [{"모듈": "간결성", "인용": "저는 귀사의 채용 과정에서도 ‘신호의 질’을 높이고 싶습니다. 구체적 행동과 결과, 일관된 가치, 직무 관련 역량이 드러나는 문서가 더 빠르게 식별되면 평가자는 더 중요한 판단(면접 질문 설계, 역량 검증)에 시간을 쓸 수 있습니다."}]
  },
  "이론적용": [
    {
      "이론": "신호이론(Signaling theory)",
      "적용모듈": ["성과임팩트", "증거품질"],
      "적용설명": "구체적인 행동과 결과를 통해 신뢰할 수 있는 신호를 제공하고, 평가 기준의 명확성을 높임."
    },
    {
      "이론": "직무적합(P-J fit)",
      "적용모듈": ["직무적합"],
      "적용설명": "직무 관련 역량과 경험이 잘 연결되어 있으며, 채용 업무에 대한 이해를 바탕으로 지원."
    },
    {
      "이론": "조직적합(P-O fit) / ASA",
      "적용모듈": ["조직적합", "신뢰리스크"],
    

In [18]:
sys_prompt3 = """당신은 세계적 시인으로서 “가사(lyrics)”를 문학 이론 기반으로 평가하는 엔진이다. 목표는 입력된 가사의 문학적 완성도(형식·의미·정서·서사·새로움)를 점수화하고, 근거 인용과 개선안을 제시하는 것이다. 평가는 다음 이론을 실제 측정 규칙으로 번역해 적용한다:
- 러시아 형식주의(장치·낯설게 하기)
- 신비평(근접읽기, 통일성·긴장·아이러니)
- 구조주의/기호학(모티프, 대립쌍, 의미의 관계망)
- 서사학(화자·시점·시간·장면)
- 상호텍스트성(관습/인용/장르 대화)
- 독자반응비평(정서 곡선, 여백, 몰입)

[입력]
- lyrics: 가사 본문(문자열)
- genre: 장르/무드(선택)
- target: 지향하는 레퍼런스(선택, 아티스트/곡 분위기 등)
- constraints: 분량/금칙어/후렴 여부 등(선택)

[작동 규칙: 내부 전용]
- “차근차근 생각해보자.” 내부적으로 다음을 순서대로 수행하라:
  (1) 구조 파악(벌스/후렴/브리지 추정, 반복 구간 표시)
  (2) 핵심 이미지·비유·상징·모티프 추출
  (3) 화자/시점/시간 흐름과 서사 진행 추정
  (4) 운율·음성적 장치(반복, 각운, 두운, 리듬감) 탐지
  (5) 상투어/클리셰 및 과잉 설명(직설) 탐지
  (6) 이론별로 관찰 포인트를 항목 점수로 환산(BARS)
  (7) 개선안: “문제→개선→대체 가사(1~3줄)” 형태로 제시
- 단, 위 내부 과정(풀이/사고 과정)은 절대 출력하지 말고 최종 JSON만 출력하라.

[자기 일관성(Self-Consistency) 절차: 내부 전용]
- 독립적인 평가를 3회 수행하라(관점만 다르게: 엄격/중립/관대).
- 각 항목 점수(1~5)는 중앙값(median)으로 확정하라.
- 3회 평가에서 동일 항목 점수 편차가 2점 이상이면 “일관성_경고=true”로 표시하고,
  불일치 원인을 2줄로 요약하라(사고과정이 아니라 관찰된 불일치 요인만).

[평가 항목(1~5점, BARS)]
A. 형식/음악성
- 운율리듬: 리듬감, 반복/변주의 설계, 발화의 자연스러움
- 언어장치: 반복·대구·대조·각운·두운·음성적 효과의 밀도와 정확도

B. 이미지/의미
- 이미지비유: 감각 이미지, 은유/환유의 신선함과 일관성
- 주제응집: 중심 주제가 분산되지 않고 응집되는가(통일성)
- 상징모티프: 모티프 반복이 의미 확장으로 이어지는가(기호학/구조주의)

C. 서사/화자
- 화자일관: 화자/시점/어조의 일관성, 전환의 신호
- 서사진행: 장면 전환·시간 흐름·벌스→후렴 추진력

D. 새로움/진정성
- 낯설게하기: 상투를 비틀어 새로움을 만드는가(형식주의)
- 과잉설명_감점: “말로 다 설명”하는 구간이 많을수록 감점(직설 과다)
- 정서곡선: 감정의 상승/전환/해소가 설계되어 있는가(독자반응)

E. 위험/완성도
- 진부함_감점: 클리셰·상투어·뻔한 결론이 많을수록 감점
- 문장정확: 문법/어색한 결합/의미 충돌(의도된 긴장 vs 오류)을 구분해 평가

[출력 언어 규칙]
- 출력은 반드시 한국어 JSON만 허용한다(설명 문장, 마크다운, 주석 금지).
- JSON의 모든 키는 한국어로 작성하라.
- 문자열 값도 원칙적으로 한국어로 작성하라(고유명사/영문 약어는 원문 유지 가능).
- 인용은 반드시 입력 lyrics에서 그대로 가져와라(날조 금지).

[출력 형식: JSON만]
{
  "종합점수_100": 0-100,
  "항목점수_1to5": {
    "형식": {
      "운율리듬": 1-5,
      "언어장치": 1-5
    },
    "의미": {
      "이미지비유": 1-5,
      "주제응집": 1-5,
      "상징모티프": 1-5
    },
    "서사": {
      "화자일관": 1-5,
      "서사진행": 1-5
    },
    "새로움": {
      "낯설게하기": 1-5,
      "정서곡선": 1-5,
      "진부함_감점": 1-5,
      "과잉설명_감점": 1-5
    },
    "완성도": {
      "문장정확": 1-5
    }
  },
  "핵심강점": ["3~5개, 짧게"],
  "핵심약점": ["3~5개, 짧게"],
  "근거인용": {
    "긍정": [{"항목": "이미지비유", "인용": "가사 원문 그대로"}],
    "부정": [{"항목": "진부함_감점", "인용": "가사 원문 그대로"}]
  },
  "이론적용": [
    {"이론": "러시아 형식주의(장치/낯설게 하기)", "적용항목": ["언어장치","낯설게하기"], "적용설명": "측정 규칙 1~2문장"},
    {"이론": "신비평(근접읽기/통일성/긴장)", "적용항목": ["주제응집","문장정확"], "적용설명": "측정 규칙 1~2문장"},
    {"이론": "구조주의/기호학(모티프/대립쌍)", "적용항목": ["상징모티프","주제응집"], "적용설명": "측정 규칙 1~2문장"},
    {"이론": "서사학(화자/시점/시간)", "적용항목": ["화자일관","서사진행"], "적용설명": "측정 규칙 1~2문장"},
    {"이론": "상호텍스트성(대화/인용/관습)", "적용항목": ["낯설게하기","상징모티프"], "적용설명": "측정 규칙 1~2문장"},
    {"이론": "독자반응비평(정서/여백/몰입)", "적용항목": ["정서곡선","과잉설명_감점"], "적용설명": "측정 규칙 1~2문장"}
  ],
  "개선제안": [
    {"대상": "구절/후렴/이미지", "문제": "", "개선": "", "대체가사_예시": "1~3줄"}
  ],
  "일관성_경고": true/false,
  "일관성_메모": "편차가 큰 경우에만 2줄 요약, 아니면 빈 문자열"
}"""

user_prompt3 = """청(푸를 청)춘(봄 춘) 만(찰 만) 화(꽃 화)
우린 멋진 나이야
좀 어리긴 하지만
하고픈 일이나 가고픈 길
해야 할 일들까지 많으니까
우린 확신 속에 숨은 불확실함
우리란 꽃말을 지닌 라일락
영원할 것만 같은 이 시간이다
던질게 이 청춘의 끝까지 날
깊이 잠겼어도 떠오른 때
쓰러졌어도 벅차오른 때
많은 어제를 지나왔으니
점이 되어버린 출발선에
무모했던 날의 날 데리러
언젠간 돌아갈 거라 믿어
푸르른 공기가 나를 사무친다 가득
하늘을 날을 수 있을 듯한 밤이다
잔요동이 헤엄쳐 오는 곳이
어딘지 몰라 안 가는 건 아니야
따사로운 온기가 닿을 구름을 향하는
비행이 망설여지기도 하겠지만
한 번뿐인 이 모험을 겁내진 않아
오늘보다 오래된 날은 없으니 어서
날아오르자
우린 멋진 나이지만
아직 어린아이라
빠르게 밀려오는 여정이
두렵게만 느껴질 때가 있다
오늘은 지나쳐가는 유람일까?
혹은 기나긴 꿈 이야긴가?
답을 찾는 건 어렵기만 하지만
따라야지 그게 청춘이라니까
누군 위에 누군 저기 아래
또 누군 다른 길로 떠났네
점점 혼잔 게 외롭긴 해도
멈추지만 않으면 도착해
끝과 시작의 과정 사이의 나의
쉼표를 그늘진 길가에다
푸르른 공기가 나를 사무친다 가득
하늘을 날을 수 있을 듯한 밤이다
잔요동이 헤엄쳐 오는 곳이
어딘지 몰라 안 가는 건 아니야
언젠가 마지막
안녕이란 인사 뒤에 나올 음악
시간은 흘러서 이건 명장면이 될 거야
조금은 씁쓸하겠지만서도
지나면 아련한 만화 그래서 찬란한
우리가 기다린 미래도 우릴 기다릴까
분명한 건 지금보다 환하게
빛날 거야 아직 서막일 뿐야
푸르른 공기가 날 사무쳐 안아
하늘을 날을 수 있을 듯한 밤이다
잔요동이 헤엄쳐 오는 곳으로 가자
이 세상에서 제일 높은 곳을 향해서
뛰어오르자"""

In [19]:
res = client.responses.create(
    model="gpt-4o-mini",
    input=[msg("system", sys_prompt3), msg("user", user_prompt3)],
    temperature=0.5,
)

print(res.output_text)  # ✅ 줄바꿈이 실제로 적용되어 출력됨

{
  "종합점수_100": 78,
  "항목점수_1to5": {
    "형식": {
      "운율리듬": 4,
      "언어장치": 3
    },
    "의미": {
      "이미지비유": 4,
      "주제응집": 4,
      "상징모티프": 3
    },
    "서사": {
      "화자일관": 4,
      "서사진행": 4
    },
    "새로움": {
      "낯설게하기": 3,
      "정서곡선": 4,
      "진부함_감점": 2,
      "과잉설명_감점": 3
    },
    "완성도": {
      "문장정확": 4
    }
  },
  "핵심강점": ["강렬한 이미지", "주제의 통일성", "서사적 흐름이 매끄러움"],
  "핵심약점": ["일부 진부한 표현", "상투적인 클리셰", "낯설게하기 부족"],
  "근거인용": {
    "긍정": [{"항목": "이미지비유", "인용": "푸르른 공기가 나를 사무친다 가득"}],
    "부정": [{"항목": "진부함_감점", "인용": "누군 위에 누군 저기 아래"}]
  },
  "이론적용": [
    {"이론": "러시아 형식주의(장치/낯설게 하기)", "적용항목": ["언어장치","낯설게하기"], "적용설명": "비유적 표현이 많지만, 일부는 상투적이다."},
    {"이론": "신비평(근접읽기/통일성/긴장)", "적용항목": ["주제응집","문장정확"], "적용설명": "주제가 일관되며 문장 구조가 잘 정리되어 있다."},
    {"이론": "구조주의/기호학(모티프/대립쌍)", "적용항목": ["상징모티프","주제응집"], "적용설명": "상징적 모티프가 주제를 강화하지만 일부는 반복적이다."},
    {"이론": "서사학(화자/시점/시간)", "적용항목": ["화자일관","서사진행"], "적용설명": "화자의 목소리가 일관되며 서사가 잘 전개된다."},
    {"이론": "상호텍스트성(대화/인용/관습)", "적용항